### First approach


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import bootstrap_plot
import plotly.express as px

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore")

import catboost
import xgboost
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score,make_scorer
from xgboost import XGBRegressor

In [ ]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e5/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv')
submission_data = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv')

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(submission_data.shape)

data = train_data.drop('id',axis=1)

In [ ]:
data.describe().T

In [ ]:
sns.kdeplot(data['FloodProbability'], fill=True,gridsize=100)
plt.title('FloodProbability')
plt.grid()
plt.show()

In [ ]:
features = data.keys()

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(15, 25))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.violinplot(y=data[feature], ax=axes[i])
    axes[i].set_title(feature)
    axes[i].set_xlabel('')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(15, 25))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.boxplot(y=data[feature], ax=axes[i])
    ax=axes[i].set_title(feature)
    ax=axes[i].set_xlabel('')
plt.tight_layout()
plt.show()

In [ ]:
sns.barplot(x='MonsoonIntensity', y='FloodProbability', data=data)

In [ ]:
plt.figure(figsize=(25, 25))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
Data ,_ = train_test_split(data,test_size=0.8,random_state=42)

In [ ]:
fig = px.scatter_3d(Data,x='RiverManagement',y='WetlandLoss',z='AgriculturalPractices',color='FloodProbability')
fig.show()

In [ ]:
num_cols = len(data.columns)
num_rows = int(np.ceil(num_cols / 3))  #ensure that there are enough rows to accommodate all columns

fig, axs = plt.subplots(num_rows, 3, figsize=(20, num_rows * 5))
for i, col in enumerate(data.columns):
    ax = axs[i // 3, i % 3]
    ax.hist(data[col], bins=10)
    ax.set_title(f'Histogram of {col}')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data['FloodProbability'], kde=True)
plt.title('Distribution of Flood Probability')
plt.xlabel('Flood Probability')
plt.ylabel('Frequency')
plt.grid(True, linestyle=':', alpha=0.7)
plt.show()

In [ ]:
plt.figure(figsize=(25, 10))
sns.boxplot(data=data[features])
plt.xticks(rotation=90)
plt.title('Features Boxplot',fontsize = 20)
plt.ylabel('Frequencies',fontsize = 20)
plt.show()

In [ ]:
subset_features = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
                   'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality']

sns.pairplot(Data[subset_features])
plt.title('Pairplot of Selected Features')
plt.show()

In [ ]:
Data[subset_features].plot(subplots = True)
plt.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
def plot_two_plots(data):

    num_features = len(data.columns)
    num_cols = 2
    num_rows = (num_features + 1) // num_cols

    print("Number of features:", num_features)

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, num_rows*5))
    axs = axs.flatten()

    for i, feature in enumerate(data.columns):
        try:
            sns.violinplot(y=data[feature], ax=axs[i*2])
            axs[i*2].set_title('Violinplot of ' + feature)
            sns.histplot(data=data, x=feature, kde=True, ax=axs[i*2+1])
            axs[i*2+1].set_title('Histogram of ' + feature)
        except IndexError:
            pass

    plt.tight_layout()
    plt.show()

In [ ]:
mean_values = data.mean()
plt.figure(figsize=(12, 6))
sns.barplot(x=mean_values.index, y=mean_values.values)
plt.xticks(rotation=90)
plt.title('Mean Value of Features')
plt.ylabel('Mean Value')
plt.show()

In [ ]:
bootstrap_plot(data[feature], size=50, samples=50000, color='blue')

In [ ]:
train_data['fsum'] = train_data.iloc[:, :-2].sum(axis=1)
test_data['fsum'] = test_data.iloc[:, :-1].sum(axis=1)

In [ ]:
X= train_data.drop(['FloodProbability'], axis=1)
y= train_data['FloodProbability']

In [ ]:
test = test_data.drop('id',axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
cat = CatBoostRegressor(random_seed=12,
                        iterations=1500,
                        depth=7,
                        colsample_bylevel=1.0,
                        verbose=False)

In [ ]:
cat.fit(X_train,y_train)

In [ ]:
prediction = cat.predict(test_data)

In [ ]:
submission_data.keys()

In [ ]:
id = submission_data['id']

In [ ]:
submission = pd.DataFrame({'id':id, 'FloodProbability': prediction})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission

### Second approach

In [ ]:
!pip install autogluon.tabular

import numpy as np
import pandas as pd
import os
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 858.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 632.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
data_url_train = '/kaggle/input/playground-series-s4e5/train.csv'
train_data = TabularDataset(data_url_train)
train_data.head()

In [ ]:
data_url_test = '/kaggle/input/playground-series-s4e5/test.csv'
test_data = TabularDataset(data_url_test)
test_data.head()

In [ ]:
initial_features = list(test_data.drop(columns=["id"]).columns)
initial_features

In [ ]:
unique_vals = []
for df in [train_data, test_data]:
    for col in initial_features:
        unique_vals += list(df[col].unique())

unique_vals = list(set(unique_vals))
unique_vals

In [ ]:
for df in [train_data, test_data]:
    df['fsum'] = df[initial_features].sum(axis=1)
    df['fstd'] = df[initial_features].std(axis=1)
    df['special1'] = df['fsum'].isin(np.arange(72, 76))
    df['fskew'] = df[initial_features].skew(axis=1)
    df['fkurtosis'] = df[initial_features].kurtosis(axis=1)

    for i in [0.0, 0.2, 0.4, 0.5, 0.6, 0.8, 1.0]:
        df['q_{}'.format(int(i*100))] = df[initial_features].quantile(i, axis = 1)

    for v in unique_vals:
        df['cnt_{}'.format(v)] = (df[initial_features] == v).sum(axis=1)

In [ ]:
X_train, X_test = train_test_split(
     train_data.drop(columns=["id"]), test_size=0.1, random_state=42, stratify=train_data.FloodProbability)

In [ ]:
hyperparameter_tune_kwargs = {
    'num_trials': 40,
    'scheduler' : 'local',
    'searcher'  : 'auto',
}

predictor = TabularPredictor(label = 'FloodProbability',
                             eval_metric = 'r2',
                             problem_type = "regression",
                            )
predictor.fit(X_train,
              time_limit = 11*60*60,
              hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
              presets = 'good_quality',
              save_space = True,
              keep_only_best = False,
             )

In [ ]:
predictor.evaluate(X_test)

In [ ]:
LB = predictor.leaderboard(X_test)
LB

In [ ]:
test_preds = predictor.predict(test_data.drop(columns=["id"]) )
submission = pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")
submission.FloodProbability = test_preds.values
submission.head()

In [ ]:
submission.to_csv("submission.csv",index=False)

### Third approach

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [ ]:
df_train=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
df_test=pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")
#
print("Train:",len(df_train))
sample_sub=pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")
#
df_train.head()

In [ ]:
num_cols=['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']

unique_vals = []
for df in [df_train, df_test]:
    for col in num_cols:
        unique_vals += list(df[col].unique())

unique_vals = list(set(unique_vals))
#
def getFeats(df):

    scaler = StandardScaler()

    df['ClimateAnthropogenicInteraction'] = (df['MonsoonIntensity'] + df['ClimateChange']) * (df['Deforestation'] + df['Urbanization'] + df['AgriculturalPractices'] + df['Encroachments'])
    df['InfrastructurePreventionInteraction'] = (df['DamsQuality'] + df['DrainageSystems'] + df['DeterioratingInfrastructure']) * (df['RiverManagement'] + df['IneffectiveDisasterPreparedness'] + df['InadequatePlanning'])

    df['sum'] = df[num_cols].sum(axis=1)
    df['std']  = df[num_cols].std(axis=1)
    df['mean'] = df[num_cols].mean(axis=1)
    df['max']  = df[num_cols].max(axis=1)
    df['min']  = df[num_cols].min(axis=1)
    df['mode'] = df[num_cols].mode(axis=1)[0]
    df['median'] = df[num_cols].median(axis=1)
    df['q_25th'] = df[num_cols].quantile(0.25, axis=1)
    df['q_75th'] = df[num_cols].quantile(0.75, axis=1)
    df['skew'] = df[num_cols].skew(axis=1)
    df['kurt'] = df[num_cols].kurt(axis=1)
    df['sum_72_76'] = df['sum'].isin(np.arange(72, 76))
    for i in range(10,100,10):
        df[f'{i}th'] = df[num_cols].quantile(i/100, axis=1)
    df['harmonic'] = len(num_cols) / df[num_cols].apply(lambda x: (1/x).mean(), axis=1)
    df['geometric'] = df[num_cols].apply(lambda x: x.prod()**(1/len(x)), axis=1)
    df['zscore'] = df[num_cols].apply(lambda x: (x - x.mean()) / x.std(), axis=1).mean(axis=1)
    df['cv'] = df['std'] / df['mean']
    df['Skewness_75'] = (df[num_cols].quantile(0.75, axis=1) - df[num_cols].mean(axis=1)) / df[num_cols].std(axis=1)
    df['Skewness_25'] = (df[num_cols].quantile(0.25, axis=1) - df[num_cols].mean(axis=1)) / df[num_cols].std(axis=1)
    df['2ndMoment'] = df[num_cols].apply(lambda x: (x**2).mean(), axis=1)
    df['3rdMoment'] = df[num_cols].apply(lambda x: (x**3).mean(), axis=1)
    df['entropy'] = df[num_cols].apply(lambda x: -1*(x*np.log(x)).sum(), axis=1)

    for v in unique_vals:
        if v<16:
            df['cnt_{}'.format(v)] = (df[num_cols] == v).sum(axis=1)

    df[num_cols] = scaler.fit_transform(df[num_cols])

    return df


In [ ]:
df_train['typ']=0
df_test['typ']=1
#
df_all=pd.concat([df_train,df_test],axis=0)
df_all=getFeats(df_all)
df_all.head()


In [ ]:
df_train=df_all[df_all['typ']==0]
df_test=df_all[df_all['typ']==1]
#
X=df_train.drop(['id','FloodProbability','typ'],axis=1)
y=df_train['FloodProbability']
#
feats=list(X.columns)

In [ ]:
lgb_params = {
    'boosting_type': 'gbdt',
    'n_estimators':2000,
    'learning_rate' :  0.012,
    #'device':'gpu',
    'num_leaves' : 250,
    'subsample_for_bin': 165700,
    'min_child_samples': 114,
    'reg_alpha': 2.075e-06,
    'reg_lambda': 3.839e-07,
    'colsample_bytree': 0.9634,
    'subsample': 0.9592,
    'max_depth': 10,
    'random_state':0,
    'verbosity':-1}

xgb_params ={'n_estimators':8000,
             'max_depth': 10,
             #'tree_method': 'gpu_hist',
             'learning_rate': 0.01,
             'random_state':0,
             }


cat_params = {'n_estimators':12000,
             'l2_leaf_reg': 0.0017992898021052064,
             'max_bin': 200,
             'learning_rate': 0.016714889518285515,
             'max_depth': 7,
             'random_state': 0,
             'min_data_in_leaf': 288
             }

In [ ]:
## Function for Cross Validation
def cross_val_train(X,y,df_test,params,mName):

    spl=7
    test_preds = np.zeros((len(df_test)))
    val_preds = np.zeros((len(X)))
    val_scores, train_scores = [],[]

    cv = KFold(spl, shuffle=True, random_state=42)

    for fold, (train_ind, valid_ind) in enumerate(cv.split(X,y)):

        X_train = X.iloc[train_ind]
        y_train = y[train_ind]
        X_val = X.iloc[valid_ind]
        y_val = y[valid_ind]

        if mName=='LGB':
            model = lgb.LGBMRegressor(**params)
            model.fit(X_train, y_train,
                        eval_set=[(X_val, y_val)],
                        callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)])

        if mName=='XGB':
            model = XGBRegressor(**params)
            model.fit(X_train, y_train,
                              eval_set=[(X_train, y_train), (X_val, y_val)],
                              early_stopping_rounds=50,
                              verbose=100)

        if mName=='CAT':
            model = CatBoostRegressor(**params)
            model.fit(X_train, y_train,
                              eval_set=[(X_train, y_train), (X_val, y_val)],
                              early_stopping_rounds=50,
                              verbose=500)

        y_pred_trn=model.predict(X_train)
        y_pred_val=model.predict(X_val)
        train_r2 = r2_score(y_train, y_pred_trn)
        val_r2 = r2_score(y_val, y_pred_val)
        print("Fold:",fold, " Train R2:",np.round(train_r2,5), " Val R2:",np.round(val_r2,5))

        test_preds += model.predict(df_test[feats])/spl
        val_preds[valid_ind] = model.predict(X_val)
        val_scores.append(val_r2)
        print("-"*50)

    return val_scores, val_preds, test_preds

# Evaluate the model
def modelEval(y,val_preds):
    mse = mean_squared_error(y,val_preds)
    rmse = np.sqrt(mean_squared_error(y, val_preds))
    r2 = r2_score(y, val_preds)
    #
    print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    print(f'R2: {r2}')


In [ ]:
val_scores_cat, val_preds_cat, test_preds_cat=cross_val_train(X,y,df_test,cat_params,'CAT')

In [ ]:
# Evaluate the model
modelEval(y,val_preds_cat)

In [ ]:
val_scores_lgb, val_preds_lgb, test_preds_lgb=cross_val_train(X,y,df_test,lgb_params,'LGB')

In [ ]:
# Evaluate the model
modelEval(y,val_preds_lgb)

In [ ]:
val_scores_xgb, val_preds_xgb, test_preds_xgb=cross_val_train(X,y,df_test,xgb_params,'XGB')


In [ ]:
# Evaluate the model
modelEval(y,val_preds_xgb)

In [ ]:
val_preds = val_preds_lgb*0.6 + val_preds_xgb*0.3 + val_preds_cat*0.1
test_preds = test_preds_lgb*0.6 + test_preds_xgb*0.3 + test_preds_cat*0.1

In [ ]:
# Evaluate the Ensemble
modelEval(y,val_preds)

In [ ]:
plt.figure(figsize=(12,6))
# Calculate residuals
residuals = y - val_preds
# Plot residuals
plt.scatter(val_preds, residuals)
plt.axhline(y=0, color='red', linestyle='--',linewidth=3)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

In [ ]:
# Plot histogram of residuals
plt.figure(figsize=(12,6))
plt.hist(residuals, bins=100, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()

In [ ]:
sub=sample_sub[['id']]
sub['FloodProbability'] = test_preds
sub.to_csv('submission.csv', index=False)
sub.head()